In [7]:
from scipy.stats import entropy
import nibabel as nib 
import numpy as np 
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import torch

In [8]:
#check which statistic is most useful to train model with segmentation masks
# statistics include: mean, standard deviation, variance, skewnee, kurtosis, entropy

#load in the data
hard_image = '/media/krishar1/Elements1/AnatomyConstrainedMultipathGAN/B30f_B50f/hard_masked/118866/118866.nii.gz'
soft_image = '/media/krishar1/Elements1/AnatomyConstrainedMultipathGAN/B30f_B50f/soft_masked/118866/118866.nii.gz'
hard_img = nib.load(hard_image)
soft_img = nib.load(soft_image)
hard_data = hard_img.get_fdata()
soft_data = soft_img.get_fdata()

#Clip and normalize the image 
clipped_hard = np.clip(hard_data, -1024, 3072)
clipped_soft = np.clip(soft_data, -1024, 3072)
normalizer = interp1d([-1024,3072], [-1,1])
norm_data_hard = normalizer(clipped_hard)
norm_data_soft = normalizer(clipped_soft)

#calculate the statistics
hard_mean = np.mean(norm_data_hard)
soft_mean = np.mean(norm_data_soft)
hard_mean, soft_mean

(-0.78437651960053, -0.7854122403153297)

In [9]:
hard_slice = nib.load("/home-local/Kernel_Conversion/AnatomyConstrainedMultipathGAN/AnatomyconstrainedMultipathGAN/training_data/train_siemens_masked_hard/118596_100.nii.gz")
soft_slice = nib.load("/home-local/Kernel_Conversion/AnatomyConstrainedMultipathGAN/AnatomyconstrainedMultipathGAN/training_data/train_siemens_masked_soft/118596_100.nii.gz")
hard_mask_slice = nib.load("/home-local/Kernel_Conversion/AnatomyConstrainedMultipathGAN/AnatomyconstrainedMultipathGAN/training_data/siemens_hard_mask_slices/118596_100.nii.gz")
soft_mask_slice = nib.load("/home-local/Kernel_Conversion/AnatomyConstrainedMultipathGAN/AnatomyconstrainedMultipathGAN/training_data/siemens_soft_mask_slices/118596_100.nii.gz")

hard_slice_data = hard_slice.get_fdata()[:,:,0]
soft_slice_data = soft_slice.get_fdata()[:,:,0]
hard_mask_slice_data = hard_mask_slice.get_fdata()[:,:,0]
soft_mask_slice_data = soft_mask_slice.get_fdata()[:,:,0]

normalizer = interp1d([-1024,3072], [-1,1])
clipped_hard = np.clip(hard_slice_data, -1024, 3072)
clipped_soft = np.clip(soft_slice_data, -1024, 3072)
norm_data_hard = normalizer(clipped_hard)
norm_data_soft = normalizer(clipped_soft)

#Check the unique values in the mask
np.unique(hard_mask_slice_data), np.unique(soft_mask_slice_data)

(array([ 0.,  1.,  2.,  3.,  4.,  7.,  8., 14., 15., 16., 18., 25., 26.]),
 array([ 0.,  1.,  2.,  3.,  4.,  7.,  8., 14., 15., 16., 18., 25., 26.]))

In [10]:
#For every label in the mask, calculate the mean for the hard and soft image 
hard_means = []
soft_means = []
hard_std = []
soft_std = []
for label in np.unique(hard_mask_slice_data):
    if label == 0:
        continue
    hard_means.append(np.mean(norm_data_hard[hard_mask_slice_data == label]))
    soft_means.append(np.mean(norm_data_soft[soft_mask_slice_data == label]))
    hard_std.append(np.std(norm_data_hard[hard_mask_slice_data == label]))
    soft_std.append(np.std(norm_data_soft[soft_mask_slice_data == label]))

#Sum hard and soft means
hard_sum_mean = np.sum(hard_means)
soft_sum_mean = np.sum(soft_means)

hard_sum_std = np.sum(hard_std)
soft_sum_std = np.sum(soft_std)
hard_sum_mean, soft_sum_mean, hard_sum_std, soft_sum_std

(-6.002999211840307,
 -6.00300900509957,
 0.000497463598008678,
 0.0003762428362173199)

In [11]:
torch.unique(torch.tensor(hard_mask_slice_data)), torch.unique(torch.tensor(soft_mask_slice_data))

(tensor([ 0.,  1.,  2.,  3.,  4.,  7.,  8., 14., 15., 16., 18., 25., 26.],
        dtype=torch.float64),
 tensor([ 0.,  1.,  2.,  3.,  4.,  7.,  8., 14., 15., 16., 18., 25., 26.],
        dtype=torch.float64))

In [12]:
#Convert numpy array to a pytroch tensor of size (1,1,512,512)
hard_tensor = torch.tensor(norm_data_hard).unsqueeze(0).unsqueeze(0)
soft_tensor = torch.tensor(norm_data_soft).unsqueeze(0).unsqueeze(0)
hard_mask_tensor = torch.tensor(hard_mask_slice_data).unsqueeze(0).unsqueeze(0)
soft_mask_tensor = torch.tensor(soft_mask_slice_data).unsqueeze(0).unsqueeze(0)

#Masks have multiple label, compute mean for each label
hard_means = []
soft_means = []
hard_std = []
soft_std = []
for label in np.unique(hard_mask_tensor):
    if label == 0:
        continue
    hard_means.append(torch.mean(hard_tensor[hard_mask_tensor == label]))
    soft_means.append(torch.mean(soft_tensor[soft_mask_tensor == label]))
    hard_std.append(torch.std(hard_tensor[hard_mask_tensor == label]))
    soft_std.append(torch.std(soft_tensor[soft_mask_tensor == label]))



In [13]:
hard_sum_mean = torch.sum(torch.tensor(hard_means))
soft_sum_mean = torch.sum(torch.tensor(soft_means))
hard_sum_std = torch.sum(torch.tensor(hard_std))
soft_sum_std = torch.sum(torch.tensor(soft_std))

hard_sum_mean, soft_sum_mean, hard_sum_std, soft_sum_std

(tensor(-6.0030, dtype=torch.float64),
 tensor(-6.0030, dtype=torch.float64),
 tensor(0.0005, dtype=torch.float64),
 tensor(0.0004, dtype=torch.float64))

In [14]:
torch.unique(hard_mask_tensor), torch.unique(soft_mask_tensor)

(tensor([ 0.,  1.,  2.,  3.,  4.,  7.,  8., 14., 15., 16., 18., 25., 26.],
        dtype=torch.float64),
 tensor([ 0.,  1.,  2.,  3.,  4.,  7.,  8., 14., 15., 16., 18., 25., 26.],
        dtype=torch.float64))

In [1]:
masks = ["lung_lower_lobe_left.nii.gz", "lung_lower_lobe_right.nii.gz", "lung_upper_lobe_left.nii.gz", "lung_upper_lobe_right.nii.gz", "lung_middle_lobe_right.nii.gz",
         "skeletal_muscle.nii.gz", "subcutaneous_fat.nii.gz", "trachea.nii.gz", "liver.nii.gz", "heart.nii.gz", "aorta.nii.gz", "kidney_left.nii.gz", "kidney_right.nii.gz",
         "pancreas.nii.gz", "common_carotid_artery_left.nii.gz", "common_carotid_artery_right.nii.gz", "inferior_vena_cava.nii.gz", "pulmonary_vein.nii.gz", 
         "subclavian_artery_left.nii.gz", "subclavian_artery_right.nii.gz", "superior_vena_cava.nii.gz", "rib_left_1.nii.gz", "rib_left_2.nii.gz", "rib_left_3.nii.gz",
         "rib_left_4.nii.gz", "rib_left_5.nii.gz", "rib_left_6.nii.gz", "rib_left_7.nii.gz", "rib_left_8.nii.gz", "rib_left_9.nii.gz", "rib_left_10.nii.gz",
         "rib_left_11.nii.gz", "rib_left_12.nii.gz", "rib_right_1.nii.gz", "rib_right_2.nii.gz", "rib_right_3.nii.gz", "rib_right_4.nii.gz", "rib_right_5.nii.gz",
         "rib_right_6.nii.gz", "rib_right_7.nii.gz", "rib_right_8.nii.gz", "rib_right_9.nii.gz", "rib_right_10.nii.gz", "rib_right_11.nii.gz", "rib_right_12.nii.gz", "spleen.nii.gz", 
         "costal_cartilages.nii.gz", "adrenal_gland_left.nii.gz", "adrenal_gland_right.nii.gz", "colon.nii.gz", "clavicula_left.nii.gz", "clavicula_right.nii.gz", "duodenum.nii.gz", 
         "gallbladder.nii.gz", "portal_vein_and_splenic_vein.nii.gz", "scapula_left.nii.gz", "scapula_right.nii.gz", "sternum.nii.gz", "brachiocephalic_trunk.nii.gz", "brachiocephalic_vein_left.nii.gz",
         "brachiocephalic_vein_right.nii.gz", "vertebrae_T1.nii.gz", "vertebrae_T2.nii.gz", "vertebrae_T3.nii.gz", "vertebrae_T4.nii.gz", "vertebrae_T5.nii.gz", "vertebrae_T6.nii.gz",
         "vertebrae_T7.nii.gz", "vertebrae_T8.nii.gz", "vertebrae_T9.nii.gz", "vertebrae_T10.nii.gz", "vertebrae_T11.nii.gz", "vertebrae_T12.nii.gz"]

#convert to dictionary with index as key and mask as value
mask_dict = {}
for i, mask in enumerate(masks):
    mask_dict[i] = mask

mask_dict

{0: 'lung_lower_lobe_left.nii.gz',
 1: 'lung_lower_lobe_right.nii.gz',
 2: 'lung_upper_lobe_left.nii.gz',
 3: 'lung_upper_lobe_right.nii.gz',
 4: 'lung_middle_lobe_right.nii.gz',
 5: 'skeletal_muscle.nii.gz',
 6: 'subcutaneous_fat.nii.gz',
 7: 'trachea.nii.gz',
 8: 'liver.nii.gz',
 9: 'heart.nii.gz',
 10: 'aorta.nii.gz',
 11: 'kidney_left.nii.gz',
 12: 'kidney_right.nii.gz',
 13: 'pancreas.nii.gz',
 14: 'common_carotid_artery_left.nii.gz',
 15: 'common_carotid_artery_right.nii.gz',
 16: 'inferior_vena_cava.nii.gz',
 17: 'pulmonary_vein.nii.gz',
 18: 'subclavian_artery_left.nii.gz',
 19: 'subclavian_artery_right.nii.gz',
 20: 'superior_vena_cava.nii.gz',
 21: 'rib_left_1.nii.gz',
 22: 'rib_left_2.nii.gz',
 23: 'rib_left_3.nii.gz',
 24: 'rib_left_4.nii.gz',
 25: 'rib_left_5.nii.gz',
 26: 'rib_left_6.nii.gz',
 27: 'rib_left_7.nii.gz',
 28: 'rib_left_8.nii.gz',
 29: 'rib_left_9.nii.gz',
 30: 'rib_left_10.nii.gz',
 31: 'rib_left_11.nii.gz',
 32: 'rib_left_12.nii.gz',
 33: 'rib_right_1.nii.g